# Project 2

## Report

### Terminal

`cp ~/w205/course-content//08-Querying-Data/docker-compose.yml .`
This command is used to copy the docker-compose.yml file from the week 8 course content folder to the current directory. The “cp” is the copy command, the following address is the file path, and the “.” is the copy destination (the current folder). 


`docker-compose up -d`
This command is used to set up docker in the Google Cloud environment. The “-d” command runs docker in the background. 


`docker-compose exec kafka kafka-topics --create --topic proj2 --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181`
This command creates a Kafka topic called “proj2”. The “docker-compose” part signifies all instances of all services (Zookeeper, Cloudera, etc) running in the container, the “exec” command runs the following command inside the current container. The “Kafka Kafka-topics” indicates that the following commands will concern a Kafka topic, which is Kafka's way of organizing all of the streamed records. The “--create --topic proj2” creates a topic called proj2, while the “--partitions 1” creates one partition. Partitions are used to compute the records in parallel, the number of partitions indicates how to split data between multiple cluster nodes (called brokers in Kafka). The “--replication-factor 1” indicates the level of replication or copies that are distributed across the cluster. The “--if-not-exists” command ensures that the topic being created does not exist. The “--zookeeper” command identifies the zookeeper port that will be used to manage the communication between the brokers. 


`docker-compose exec mids bash -c "cat /w205/project-2-Marcus-M1999/datafile.json | jq '.[]' -c | kafkacat -P -b kafka:29092 -t proj2"`
This command loads the JSON data into the Kafka topic. The “mids bash -c” tells the mids container to take in the following multiple commands in the command line. The “cat” and following file address is the location of the JSON file, while the jq.[] separates each item in the JSON into a different line. Kafkacat is a command-line tool that reads messages from the specified Kafka broker, in this case, “29092”, from the topic (-t) proj2. The “-P” command to set the producer (the object that writes the events to Kafka) to the Kafka: 29092 containers with the topic proj2.


`docker-compose exec Cloudera Hadoop fs -ls /tmp/`
This command examines the Cloudera container that hosts the messages exported from Pyspark. The -ls /tmp/ command specifies the home directory of the container. 


### docker-compose.yml

The docker-compose.yml file contains the configuration for docker to run off, and all of the services as well as ports needed to run the services. From top to bottom the services include:

##### 1.)  Zookeeper
Zookeeper is the service used to manage all other services in the container. The image line refers to the version of the service being used in the container. The client port refers to the port in use to communicate with Kafka. The exposed, or open ports are listed under "expose" (for additional connections in the future).

##### 2.) Kafka 
Kafka is the second service in the docker configuration file and has several new lines including depends_on. The depends_on keyword means that docker will first spin up Zookeeper and then Kafka since Kafka uses Zookeeper to manage all other services. Under the environment variables, `KAFKA_BROKER_ID` is assigned to 1, since it's the first and only node in the cluster. (The Broker ID identifies each broker in the cluster with a unique identifier, 1 in this case.) `KAFKA_ZOOKEEPER_CONNECT` displays the port that zookeeper is connected to. `KAFKA_ADVERTISED_LISTENERS` lists all listeners that the broker (Kafka node) will display to all producers and consumers of records in the pipeline. The port connected, `kafka:29092` refers to the Kafka port that Kafka connects to communicate with Zookeeper. `KAFKA_OFFSETS_TOPIC_REPLICATOIN_FACTOR` refers to the number of replications that occur with the data between multiple brokers. In this case, the replication factor is 1 since there is only 1 broker. 

##### 3.) Cloudera
Cloudera is a hosting service for Hadoop and contains a variety of additional tools on top of Apache's version of Hadoop (Apache is the creator of Hadoop). 

##### 4.) Spark
Spark is the service used to process messages and relay them to the hdfs nodes ran by Cloudera. `stdin_open` and `TTY keep the container running in a "detached mode" allowing the service to persist in an interactive virtual environment. The `volumes` connect the directory ~/w205/ to /w205, to read and write files in spark. The `environment lists the environment variables including the `Hadoops_NameNode` cluster that spark sends the processed data to. 

##### 5.) mids
mids is the final container in the docker-compose file that holds tools such as Kafkacat (a tool used to produce and consume messages with Kafka). 


### Spark

`docker-compose exec spark pyspark`
This command spins up an instance of pyspark. 
 
`raw_data = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","proj2").option("startingOffsets", "earliest").option("endingOffsets", "latest").load()`
This command reads the data into spark by instructing pyspark that the format is a Kafka message, and the port is 29092. The spark pipeline is “subscribing” or reading data from the “proj2” topic, starting at the “earliest” or starting offset (also known as the first message), and going until the last message or the latest of the ending offset. 

`raw_data.show()`
This command reveals the raw data, since the data had not been converted to strings it did not show a tremendous amount of useful information. 
`
+----+--------------------+-----+---------+------+--------------------+-------------+
| key|               value|topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----+---------+------+--------------------+-------------+
|null|[7B 22 6B 65 65 6...|proj2|        0|     0|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|proj2|        0|     1|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|proj2|        0|     2|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|proj2|        0|     3|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|proj2|        0|     4|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|proj2|        0|     5|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|proj2|        0|     6|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|proj2|        0|     7|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|proj2|        0|     8|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|proj2|        0|     9|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|proj2|        0|    10|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|proj2|        0|    11|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|proj2|        0|    12|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|proj2|        0|    13|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|proj2|        0|    14|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|proj2|        0|    15|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|proj2|        0|    16|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|proj2|        0|    17|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|proj2|        0|    18|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|proj2|        0|    19|1969-12-31 23:59:...|            0|
+----+--------------------+-----+---------+------+--------------------+-------------+
only showing top 20 rows 
`


`data_as_strings=raw_data.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")`
This command casts the raw data as strings, both the keys and values.

`data_as_strings.cache()`
This command caches the data_as_strings dataset in memory so it can be accessed faster. 

`data_as_strings.show()`
This command displays the data in the strings data frame. Since JSON files are different from CSV files or SQL tables it did not display a lot of useful information causing me to use the following command to get a closer look at the data.

`data_as_strings.select('value').take(1)[0].value`
This command displays the first row and first entry in the JSON dataset. From this, I was able to see the nested nature of the nested field “sequences”. There are multiple fields including the questions, and count of questions, correct, incorrect, and incomplete.
`
'{"keen_timestamp":"1516717442.735266","max_attempts":"1.0","started_at":"2018-01-23T14:23:19.082Z","base_exam_id":"37f0a30a-7464-11e6-aa92-a8667f27e5dc","user_exam_id":"6d4089e4-bde5-4a22-b65f-18bce9ab79c8","sequences":{"questions":[{"user_incomplete":true,"user_correct":false,"options":[{"checked":true,"at":"2018-01-23T14:23:24.670Z","id":"49c574b4-5c82-4ffd-9bd1-c3358faf850d","submitted":1,"correct":true},{"checked":true,"at":"2018-01-23T14:23:25.914Z","id":"f2528210-35c3-4320-acf3-9056567ea19f","submitted":1,"correct":true},{"checked":false,"correct":true,"id":"d1bf026f-554f-4543-bdd2-54dcf105b826"}],"user_submitted":true,"id":"7a2ed6d3-f492-49b3-b8aa-d080a8aad986","user_result":"missed_some"},{"user_incomplete":false,"user_correct":false,"options":[{"checked":true,"at":"2018-01-23T14:23:30.116Z","id":"a35d0e80-8c49-415d-b8cb-c21a02627e2b","submitted":1},{"checked":false,"correct":true,"id":"bccd6e2e-2cef-4c72-8bfa-317db0ac48bb"},{"checked":true,"at":"2018-01-23T14:23:41.791Z","id":"7e0b639a-2ef8-4604-b7eb-5018bd81a91b","submitted":1,"correct":true}],"user_submitted":true,"id":"bbed4358-999d-4462-9596-bad5173a6ecb","user_result":"incorrect"},{"user_incomplete":false,"user_correct":true,"options":[{"checked":false,"at":"2018-01-23T14:23:52.510Z","id":"a9333679-de9d-41ff-bb3d-b239d6b95732"},{"checked":false,"id":"85795acc-b4b1-4510-bd6e-41648a3553c9"},{"checked":true,"at":"2018-01-23T14:23:54.223Z","id":"c185ecdb-48fb-4edb-ae4e-0204ac7a0909","submitted":1,"correct":true},{"checked":true,"at":"2018-01-23T14:23:53.862Z","id":"77a66c83-d001-45cd-9a5a-6bba8eb7389e","submitted":1,"correct":true}],"user_submitted":true,"id":"e6ad8644-96b1-4617-b37b-a263dded202c","user_result":"correct"},{"user_incomplete":false,"user_correct":true,"options":[{"checked":false,"id":"59b9fc4b-f239-4850-b1f9-912d1fd3ca13"},{"checked":false,"id":"2c29e8e8-d4a8-406e-9cdf-de28ec5890fe"},{"checked":false,"id":"62feee6e-9b76-4123-bd9e-c0b35126b1f1"},{"checked":true,"at":"2018-01-23T14:24:00.807Z","id":"7f13df9c-fcbe-4424-914f-2206f106765c","submitted":1,"correct":true}],"user_submitted":true,"id":"95194331-ac43-454e-83de-ea8913067055","user_result":"correct"}],"attempt":1,"id":"5b28a462-7a3b-42e0-b508-09f3906d1703","counts":{"incomplete":1,"submitted":4,"incorrect":1,"all_correct":false,"correct":2,"total":4,"unanswered":0}},"keen_created_at":"1516717442.735266","certification":"false","keen_id":"5a6745820eb8ab00016be1f1","exam_name":"Normal Forms and All That Jazz Master Class"}'
`


`import json`
This command imports the JSON library, which I used to further examine the data. 

`first_entry =json.loads(data_as_strings.select('value').take(1)[0].value)`
This command uses json.loads to load the selected data entry (in this case the first entry).

`json.loads(data_as_strings.select('value').take(2)[0].value)`
This shows the first entry on the second row. 
`
{'keen_timestamp': '1516717442.735266', 'max_attempts': '1.0', 'started_at': '2018-01-23T14:23:19.082Z', 'base_exam_id': '37f0a30a-7464-11e6-aa92-a8667f27e5dc', 'user_exam_id': '6d4089e4-bde5-4a22-b65f-18bce9ab79c8', 'sequences': {'questions': [{'user_incomplete': True, 'user_correct': False, 'options': [{'checked': True, 'at': '2018-01-23T14:23:24.670Z', 'id': '49c574b4-5c82-4ffd-9bd1-c3358faf850d', 'submitted': 1, 'correct': True}, {'checked': True, 'at': '2018-01-23T14:23:25.914Z', 'id': 'f2528210-35c3-4320-acf3-9056567ea19f', 'submitted': 1, 'correct': True}, {'checked': False, 'correct': True, 'id': 'd1bf026f-554f-4543-bdd2-54dcf105b826'}], 'user_submitted': True, 'id': '7a2ed6d3-f492-49b3-b8aa-d080a8aad986', 'user_result': 'missed_some'}, {'user_incomplete': False, 'user_correct': False, 'options': [{'checked': True, 'at': '2018-01-23T14:23:30.116Z', 'id': 'a35d0e80-8c49-415d-b8cb-c21a02627e2b', 'submitted': 1}, {'checked': False, 'correct': True, 'id': 'bccd6e2e-2cef-4c72-8bfa-317db0ac48bb'}, {'checked': True, 'at': '2018-01-23T14:23:41.791Z', 'id': '7e0b639a-2ef8-4604-b7eb-5018bd81a91b', 'submitted': 1, 'correct': True}], 'user_submitted': True, 'id': 'bbed4358-999d-4462-9596-bad5173a6ecb', 'user_result': 'incorrect'}, {'user_incomplete': False, 'user_correct': True, 'options': [{'checked': False, 'at': '2018-01-23T14:23:52.510Z', 'id': 'a9333679-de9d-41ff-bb3d-b239d6b95732'}, {'checked': False, 'id': '85795acc-b4b1-4510-bd6e-41648a3553c9'}, {'checked': True, 'at': '2018-01-23T14:23:54.223Z', 'id': 'c185ecdb-48fb-4edb-ae4e-0204ac7a0909', 'submitted': 1, 'correct': True}, {'checked': True, 'at': '2018-01-23T14:23:53.862Z', 'id': '77a66c83-d001-45cd-9a5a-6bba8eb7389e', 'submitted': 1, 'correct': True}], 'user_submitted': True, 'id': 'e6ad8644-96b1-4617-b37b-a263dded202c', 'user_result': 'correct'}, {'user_incomplete': False, 'user_correct': True, 'options': [{'checked': False, 'id': '59b9fc4b-f239-4850-b1f9-912d1fd3ca13'}, {'checked': False, 'id': '2c29e8e8-d4a8-406e-9cdf-de28ec5890fe'}, {'checked': False, 'id': '62feee6e-9b76-4123-bd9e-c0b35126b1f1'}, {'checked': True, 'at': '2018-01-23T14:24:00.807Z', 'id': '7f13df9c-fcbe-4424-914f-2206f106765c', 'submitted': 1, 'correct': True}], 'user_submitted': True, 'id': '95194331-ac43-454e-83de-ea8913067055', 'user_result': 'correct'}], 'attempt': 1, 'id': '5b28a462-7a3b-42e0-b508-09f3906d1703', 'counts': {'incomplete': 1, 'submitted': 4, 'incorrect': 1, 'all_correct': False, 'correct': 2, 'total': 4, 'unanswered': 0}}, 'keen_created_at': '1516717442.735266', 'certification': 'false', 'keen_id': '5a6745820eb8ab00016be1f1', 'exam_name': 'Normal Forms and All That Jazz Master Class'}
`


`json.loads(data_as_strings.select('value').take(8 )[0].value)`
This command shows the first entry on the eighth row. Interestingly enough this command displayed the same information as the above command.
`
{'keen_timestamp': '1516717442.735266', 'max_attempts': '1.0', 'started_at': '2018-01-23T14:23:19.082Z', 'base_exam_id': '37f0a30a-7464-11e6-aa92-a8667f27e5dc', 'user_exam_id': '6d4089e4-bde5-4a22-b65f-18bce9ab79c8', 'sequences': {'questions': [{'user_incomplete': True, 'user_correct': False, 'options': [{'checked': True, 'at': '2018-01-23T14:23:24.670Z', 'id': '49c574b4-5c82-4ffd-9bd1-c3358faf850d', 'submitted': 1, 'correct': True}, {'checked': True, 'at': '2018-01-23T14:23:25.914Z', 'id': 'f2528210-35c3-4320-acf3-9056567ea19f', 'submitted': 1, 'correct': True}, {'checked': False, 'correct': True, 'id': 'd1bf026f-554f-4543-bdd2-54dcf105b826'}], 'user_submitted': True, 'id': '7a2ed6d3-f492-49b3-b8aa-d080a8aad986', 'user_result': 'missed_some'}, {'user_incomplete': False, 'user_correct': False, 'options': [{'checked': True, 'at': '2018-01-23T14:23:30.116Z', 'id': 'a35d0e80-8c49-415d-b8cb-c21a02627e2b', 'submitted': 1}, {'checked': False, 'correct': True, 'id': 'bccd6e2e-2cef-4c72-8bfa-317db0ac48bb'}, {'checked': True, 'at': '2018-01-23T14:23:41.791Z', 'id': '7e0b639a-2ef8-4604-b7eb-5018bd81a91b', 'submitted': 1, 'correct': True}], 'user_submitted': True, 'id': 'bbed4358-999d-4462-9596-bad5173a6ecb', 'user_result': 'incorrect'}, {'user_incomplete': False, 'user_correct': True, 'options': [{'checked': False, 'at': '2018-01-23T14:23:52.510Z', 'id': 'a9333679-de9d-41ff-bb3d-b239d6b95732'}, {'checked': False, 'id': '85795acc-b4b1-4510-bd6e-41648a3553c9'}, {'checked': True, 'at': '2018-01-23T14:23:54.223Z', 'id': 'c185ecdb-48fb-4edb-ae4e-0204ac7a0909', 'submitted': 1, 'correct': True}, {'checked': True, 'at': '2018-01-23T14:23:53.862Z', 'id': '77a66c83-d001-45cd-9a5a-6bba8eb7389e', 'submitted': 1, 'correct': True}], 'user_submitted': True, 'id': 'e6ad8644-96b1-4617-b37b-a263dded202c', 'user_result': 'correct'}, {'user_incomplete': False, 'user_correct': True, 'options': [{'checked': False, 'id': '59b9fc4b-f239-4850-b1f9-912d1fd3ca13'}, {'checked': False, 'id': '2c29e8e8-d4a8-406e-9cdf-de28ec5890fe'}, {'checked': False, 'id': '62feee6e-9b76-4123-bd9e-c0b35126b1f1'}, {'checked': True, 'at': '2018-01-23T14:24:00.807Z', 'id': '7f13df9c-fcbe-4424-914f-2206f106765c', 'submitted': 1, 'correct': True}], 'user_submitted': True, 'id': '95194331-ac43-454e-83de-ea8913067055', 'user_result': 'correct'}], 'attempt': 1, 'id': '5b28a462-7a3b-42e0-b508-09f3906d1703', 'counts': {'incomplete': 1, 'submitted': 4, 'incorrect': 1, 'all_correct': False, 'correct': 2, 'total': 4, 'unanswered': 0}}, 'keen_created_at': '1516717442.735266', 'certification': 'false', 'keen_id': '5a6745820eb8ab00016be1f1', 'exam_name': 'Normal Forms and All That Jazz Master Class'}
`


`import sys`
This command imports the system library to give tools for converting JSON to SQL.

`sys.stdout = open(sys.stdout.fileno(), mode='w', encoding='utf8', buffering=1)`
This encodes the data from binary to utf8. The sys.stdout prints the strings at utf8. 

`from pyspark.sql import Row`
This is the Non-deprecated method of organizing data into a SQL table. 

`extracted_data = data_as_strings.rdd.map(lambda x: json.loads(x.value)).toDF()`
Flattens the first layer of the JSON dataset. Although the first layer is not flat (the values are no longer a subfield of the current field), the sequences field is still heavily nested and needs to be flattened. The “rdd.map” maps the lambda function to each element in the RDD (Resilient Distributed Datasets, the immutable data structure used in spark). 


`extracted_data.cache()`
This command caches the extracted_data dataset to the memory for faster access and to avoid errors in spark. 

`extracted_data.write.parquet("/tmp/extracted_data_edu")`
I wrote this data to the temporary home directory in hdfs. The parquet in this command specifies the file type as a parquet file. The address is the location of the parquet file, compressed with a library called “snappy”. 

`extracted_data.registerTempTable('ed_data')`
This command creates a table in the form of a SQL data frame called “ed_data” that I can use to query the data and extract insights. 

`def extract_details_from_json(row):
    student = json.loads(row.value)
    qs = []
    if "sequences" in student.keys():
        if "questions" in student["sequences"].keys():
            questions = student["sequences"]["questions"]
    for i in qs:
        student_details = {"question_id": i["id"],
                           "question_incomplete": 100,
                           "question_correct": 100,
                           "question_submitted": 100,
                           "base_exam_id": student["base_exam_id"],
                           "exam_name": student["exam_name"],
                           "user_exam_id": student["user_exam_id"],
                           "started_time": student["started_at"]["ratings_count"]}
        if "user_incomplete" in q.keys():
            student_details["question_incomplete"] = i["user_incomplete"]
        if "user_correct" in q.keys():
            student_details["question_correct"] = i["user_correct"]
        if "user_submitted" in q.keys():
            student_details["question_submitted"] = i["user_submitted"]
        qs.append(Row(**student_details))
    return qs`
The above function is something that I was working on to unnest the sequences data frame. I attempted to do this after looking at other examples that were posted by instructors and students. Meer Wu from our section helped me understand and guided me to this solution to flatten the sequences data frame. Unfortunately, I encountered errors with the rdd, and after testing the function out on different forms of our dataset I was unable to unnest the field. 

`spark.sql("SELECT sequences.questions.user_incomplete FROM ed_data").show()`
This effort led to the final attempt which was to manually unnest it, going layer by layer, but I encountered errors. This is because while some entries may exist others will not, creating an issue in any kind of broad operations done to the field. Note: I am using the show() function since spark uses "lazy evaluation" meaning that it will not calculate and display the information unless it has to (as indicated by the show() method). 


### Questions

##### 1.) What are the most popular exams?
`spark.sql("SELECT DISTINCT exam_name, COUNT(user_exam_id) AS user_count FROM ed_data GROUP BY exam_name ORDER BY user_count DESC").show()`
This query shows the most popular exams, and users, most popular exams are: “Learning Git”, “Introduction to Python”, “Introduction to Java”
`
+--------------------+----------+                                               
|           exam_name|user_count|
+--------------------+----------+
|        Learning Git|       394|
|Introduction to P...|       162|
|Introduction to J...|       158|
|Intermediate Pyth...|       158|
|Learning to Progr...|       128|
|Introduction to M...|       119|
|Software Architec...|       109|
|Beginning C# Prog...|        95|
|    Learning Eclipse|        85|
|Learning Apache M...|        80|
|Beginning Program...|        79|
|       Mastering Git|        77|
|Introduction to B...|        75|
|Advanced Machine ...|        67|
|Learning Linux Sy...|        59|
|JavaScript: The G...|        58|
|        Learning SQL|        57|
|Practical Java Pr...|        53|
|    HTML5 The Basics|        52|
|   Python Epiphanies|        51|
+--------------------+----------+
only showing top 20 rows
`


##### What month is the most popular for exams?
`spark.sql("SELECT COUNT(*) AS user_count, substring(started_at, 6, 2) AS Month FROM ed_data GROUP BY SUBSTRING(started_at, 6, 2) ORDER BY user_count DESC").show()`
It appears that month 11 (November) is the least popular month. This is despite the fact that classes have only been offered for 3 months. 
`
+----------+-----+                                                              
|user_count|Month|
+----------+-----+
|      1516|   12|
|      1243|   01|
|       521|   11|
+----------+-----+
`


##### How many courses taken have a certification?
`spark.sql("SELECT SUM(CASE WHEN certification = true THEN 1 ELSE 0 END) AS count, exam_name FROM ed_data GROUP BY exam_name ORDER BY count").show()`
It looks like no courses have a certification. Certifications could be a feature that is planned to be implemented later on, which is why the column is false for all courses.  
`
+-----+--------------------+                                                    
|count|           exam_name|
+-----+--------------------+
|    0|Learning Data Mod...|
|    0|Networking for Pe...|
|    0|Introduction to J...|
|    0|Learning Apache H...|
|    0|Learning Spring P...|
|    0|Learning iPython ...|
|    0|Introduction to P...|
|    0|Learning C# Best ...|
|    0|Introduction to A...|
|    0|A Practical Intro...|
|    0|I'm a Software Ar...|
|    0|Introduction to B...|
|    0|       View Updating|
|    0|Mastering Python ...|
|    0|Intermediate C# P...|
|    0|Starting a Grails...|
|    0|Introduction to A...|
|    0|JavaScript Templa...|
|    0|Being a Better In...|
|    0|Mastering Advance...|
+-----+--------------------+
only showing top 20 rows
`
